# ChatGPT

[ChatGPT](https://chat.openai.com/) é um sistema de diálogo desenvolvido pela empresa OpenAI, que utiliza modelo de linguagem GPT-3 treinado em uma ampla variedade de dados.

Na área clínica, o chatGPT pode ser utilizado para responder perguntas sobre sintomas, doenças, tratamentos comuns, estilo de vida saudável e muito mais. Atenção: a finalidade é fornecer informações educativas e orientações gerais, e não substituir a consulta com um profissional de saúde qualificado.

Neste tutorial, vamos ver como chamar o chatGPT através de API, para realizar a tarefa de **Reconhecimento de entidades nomeadas (NER)** para extrais conceitos clínicos relevantes em textos da área da saúde.

Vamos executar tarefas:

1- ***Zero-shot***: cenário em que um modelo é capaz de realizar uma tarefa sem treinamento prévio específico para essa tarefa, usando apenas informações gerais sobre o problema.

2- ***Few-shot***: cenário em que um modelo recebe um pequeno número de exemplos ou instruções de treinamento para uma determinada tarefa e é capaz de generalizar e realizar a tarefa com base nessas informações limitadas.

Leitura recomendada: https://openai.com/research/language-models-are-few-shot-learners

### Adquirindo a *API Key*

O primeiro passo é adquirir a* API Key* para poder usar a API do chatGPT. Para isso, execute os passos:

1 - Cadastro no site da openai e confirmação do número do telefone: https://openai.com/

2 - Após realizar o cadastro, se autentique e acesse o botão "API"

3 - No canto superior direito, clique no ícone para acessar sua área administrativa e acesse "*View API keys*"

4 - Clique em "*Create new secret key*"

5 - Copie a chave criada e utilize aqui, substituindo "API_KEY"

OBSERVAÇÂO: Confira aqui os limites de uso: https://platform.openai.com/account/rate-limits

### Instalando a bilbioteca necessária

Vamos usar a biblioteca `openai` para enviar e receber as requisições para o chatGPT. Primeiramente, vamos instalar aqui no nosso ambiente do Colab.

In [ ]:
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.7 MB/s eta 0:00:00


Agora já podemos usar a biblioteca. Substitua *"API_KEY"* pela chave que você gerou.

Vamos criar um método `askAI` que recebe o texto de entrada (*"prompt"*), chama o modelo GPT-3 (*"text-davinci-003"*), recebe a resposta e printa na tela.

In [ ]:
import openai

##enter your API key here, available in the Personal > View API keys on the openai website
secret_key = "API_KEY"

openai.api_key = secret_key


In [ ]:
def askAI(prom):
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prom,
      temperature=0,
      max_tokens=1000,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0)

    print(response['choices'][0]['text'])


Agora vamos setar duas frases clínicas (fictícias) como exemplo, para usarmos no modelo.

In [ ]:
frase1="Paciente com dispneia importante aos esforços, sente dor tipo peso no peito no esforço. Recebeu tratamento com nitroglicerina sublingual e apresentou melhora."
frase2="Discreto edema mmii pricn a esquerda. Dor a movimentação do torax, nega palpitação. Em uso de insulina nph, metformina, sinvastatina 40mg 2 cp, clopidogrel 75mg."

## Zero-shot

Vamos pedir pro chatGPT extrair as entidades que ele achar relevante nessas frases. Repare que não definimos quais entidades nos interessa, deixamos a cargo do modelo.

Não vamos realizar nenhum treinamento nem enviar nenhum exemplo para ele.

In [ ]:
prompt =f"Extraia as entidades nomeadas da sentença:  {frase1}"
print(frase1)
askAI(prompt)

Paciente com dispneia importante aos esforços, sente dor tipo peso no peito no esforço. Recebeu tratamento com nitroglicerina sublingual e apresentou melhora.


- Paciente 
- Dispneia 
- Dor 
- Peso 
- Peito 
- Esforço 
- Nitroglicerina 
- Sublingual 
- Melhora


In [ ]:
prompt =f"Extraia as entidades nomeadas da sentença:  {frase2}"
print(frase2)
askAI(prompt)

Discreto edema mmii pricn a esquerda. Dor a movimentação do torax, nega palpitação. Em uso de insulina nph, metformina, sinvastatina 40mg 2 cp, clopidogrel 75mg.


- Insulina NPH
- Metformina
- Sinvastatina
- Clopidogrel


Mesmo sem ter especificado o que queríamos, ele conseguiu trazer informações interessantes na área médica.

Agora vamos especificar o que queremos que ele extraia: problemas, tratamentos,anatomia e testes.

In [ ]:
prompt =f"Extraia os conceitos de problemas, tratamentos e anatomia e testes da sentença:  {frase1}"
print(frase1)
askAI(prompt)

Paciente com dispneia importante aos esforços, sente dor tipo peso no peito no esforço. Recebeu tratamento com nitroglicerina sublingual e apresentou melhora.


Problema: Dispneia aos esforços, dor tipo peso no peito no esforço.

Tratamento: Nitroglicerina sublingual.

Anatomia: Peito.

Testes: Não especificado.


In [ ]:
prompt =f"Extraia os conceitos de problemas, tratamentos, anatomia e testes da sentença:  {frase2}"
print(frase2)
askAI(prompt)

Discreto edema mmii pricn a esquerda. Dor a movimentação do torax, nega palpitação. Em uso de insulina nph, metformina, sinvastatina 40mg 2 cp, clopidogrel 75mg.


Problema: Discreto edema mmii pricn a esquerda. Dor a movimentação do torax.

Tratamento: Uso de insulina nph, metformina, sinvastatina 40mg 2 cp, clopidogrel 75mg.

Anatomia: mmii pricn a esquerda.

Testes: Palpitação.


Agora ele já trouxe informações clínicas relevantes para nós.

## Few-shot

Na sequencia, vamos ver como fazer um leve treinamento ensinando o que queremos que ele extraia, mas com apenas 5 exemplos de entrada.

Para isso, existe um formato específico para usar via *prompt*. Um exemplo seria:

```
prompt = """Extraia os conceitos da sentença

Sentença: Ecocardio e lab .
Problema: nenhum
Teste: Ecocardio, lab
Tratamento: nenhum
Anatomia: nenhum
###
Sentença: angioplastia , implante de marcapasso ( pacte assintomatico , ecg de repouso com bav de 2 grau mobitz 2 ) , cat em agosto desse ano .
Problema: bav de 2 grau mobitz 2
Teste: ecg de repouso, cat
Tratamento: angioplastia, marcapasso
Anatomia: nenhum
###
Sentença: discreto edema mmii pricn a esquerda. nega palpitação."""

```

onde na primeira frase, indicamos o que queremos que seja executado. Quebramos a linha e enviamos a sentença de exemplo seguida com as entidades que queremos extrair, separando cada instância de treinamento com *###*.

A última sentença é a que queremos que seja realizada a inferência.

Vamos testar com as nossas frases de exemplo.

In [ ]:
prompt =f"""Extraia os conceitos da sentença:

Sentença: c # cintilografia miocardica para avaliar angina.
Problema: angina
Teste: cintilografia miocardica, avaliar
Tratamento: nenhum
Anatomia: miocardica
###
Sentença: Plastia Mitral ( Insuficiencia ) , CRM Saf - 2Mg e e Saf - 3MG ).
Problema: Insuficiencia
Teste: nenhum
Tratamento: Plastia Mitral, CRM
Anatomia: Mitral
###
Sentença: Uso : AAS 100 - 1xd ; Metoprolol 25 - 1xd ; FSM - 1xd ; Levotiroxina 175 - 1xd ; Sinva 40 - 1xd ; Fluoxetina 20 - 1xd.
Problema: nenhum
Teste: nenhum
Tratamento: AAS 100, Metoprolol 25, FSM, Levotiroxina 175, Sinva 40, Fluoxetina 20
Anatomia: nenhum
###
Sentença: Refere melhora da dispneia depois da cx porem mantem aos mdoeardos - leves esforço.
Problema: dispneia, mdoeardos - leves esforço
Teste: nenhum
Tratamento: cx
Anatomia: nenhum
###
Sentença: Discreto edema mmii pricn a esquerda, dor no peito.
Problema: Discreto edema mmii pricn a esquerda
Teste: nenhum
Tratamento: nenhum
Anatomia: mmii, peito
### Sentença: {frase1}"""
print(frase1)
askAI(prompt)


Paciente com dispneia importante aos esforços, sente dor tipo peso no peito no esforço. Recebeu tratamento com nitroglicerina sublingual e apresentou melhora.

Problema: dispneia importante aos esforços, dor tipo peso no peito
Teste: nenhum
Tratamento: nitroglicerina sublingual
Anatomia: peito


In [ ]:
prompt =f"""Extraia os conceitos da sentença:

Sentença: c # cintilografia miocardica para avaliar angina.
Problema: angina
Teste: cintilografia miocardica, avaliar
Tratamento: nenhum
Anatomia: miocardica
###
Sentença: Plastia Mitral ( Insuficiencia ) , CRM Saf - 2Mg e e Saf - 3MG ).
Problema: Insuficiencia
Teste: nenhum
Tratamento: Plastia Mitral, CRM
Anatomia: Mitral
###
Sentença: Uso : AAS 100 - 1xd ; Metoprolol 25 - 1xd ; FSM - 1xd ; Levotiroxina 175 - 1xd ; Sinva 40 - 1xd ; Fluoxetina 20 - 1xd.
Problema: nenhum
Teste: nenhum
Tratamento: AAS 100, Metoprolol 25, FSM, Levotiroxina 175, Sinva 40, Fluoxetina 20
Anatomia: nenhum
###
Sentença: Refere melhora da dispneia depois da cx porem mantem aos mdoeardos - leves esforço.
Problema: dispneia, mdoeardos - leves esforço
Teste: nenhum
Tratamento: cx
Anatomia: nenhum
###
Sentença: Discreto edema mmii pricn a esquerda, dor no peito.
Problema: Discreto edema mmii pricn a esquerda
Teste: nenhum
Tratamento: nenhum
Anatomia: mmii, peito
### Sentença: {frase2}"""
print(frase2)
askAI(prompt)


Discreto edema mmii pricn a esquerda. Dor a movimentação do torax, nega palpitação. Em uso de insulina nph, metformina, sinvastatina 40mg 2 cp, clopidogrel 75mg.

Problema: Discreto edema mmii pricn a esquerda, dor a movimentação do torax
Teste: nenhum
Tratamento: insulina nph, metformina, sinvastatina 40mg 2 cp, clopidogrel 75mg
Anatomia: mmii, torax


Pronto! Nossa API já está retornando as informações que ensinamos ao modelo neste pequeno treinamento.

OBS: A cada mensagem via *prompt*, é necessário enviar novamente os exemplos *few-shot*.